In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd '/gdrive/My Drive/Colab/proyecto_perfil/modelo'

Mounted at /gdrive
/gdrive/My Drive/Colab/proyecto_perfil/modelo


In [ ]:
!ls 

1_gptmin_compositor_musical.ipynb  minGPT


In [ ]:
%cd minGPT

/gdrive/My Drive/Colab/proyecto_perfil/modelo/minGPT


In [ ]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///gdrive/My%20Drive/Colab/proyecto_perfil/modelo/minGPT
  Running setup.py develop for minGPT


In [ ]:
# Ejecutar solo la primera vez cuando se prepara el entorno de trabajo
# !git clone https://github.com/karpathy/minGPT.git 

Cloning into 'minGPT'...
remote: Enumerating objects: 486, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 486 (delta 101), reused 160 (delta 88), pack-reused 291
Receiving objects: 100% (486/486), 1.46 MiB | 8.01 MiB/s, done.
Resolving deltas: 100% (256/256), done.


In [ ]:
!pip install  music21 8.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement 8.1.0 (from versions: none)
ERROR: No matching distribution found for 8.1.0


In [ ]:
!pip list

Package                       Version
----------------------------- ----------------------
absl-py                       1.3.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.3
aiosignal                     1.3.1
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.9
bokeh

In [ ]:
import torch
import numpy as np
import json

In [ ]:

def cargarArchivosNPY(path):
    with open(path, 'rb') as f:
        datos = np.load(f,allow_pickle=True)
    return datos 

In [ ]:
def cargarVocabulario(pathVocabulario):
    with open(pathVocabulario, "r") as fp:
        mappings = json.load(fp)
    return mappings

In [ ]:
vocab=cargarVocabulario("../../vocabulario/vocabulario2022.json")
len(vocab)

135

In [ ]:
!ls 

demo.ipynb  generate.ipynb  mingpt	     mingpt.jpg  README.md  tests
font.sf2    LICENSE	    minGPT.egg-info  projects	 setup.py


In [ ]:
datos=cargarArchivosNPY("../../archivos_numpy/partituras.npy")
datos

In [ ]:
datoPrueba= datos.tolist()
datoPrueba=np.array(datoPrueba["0"])
len(datoPrueba)

310

In [ ]:
def convertir_tokens_a_notas(vocabulario,listTokens):
  vocaRev = { i:ch for i,ch in enumerate(vocab) }
  notas=[vocaRev[t] for t in listTokens]
  return notas,vocaRev



In [ ]:
convertir_tokens_a_notas(vocab,[0,1,2,3,4,30])

In [ ]:
stoi = { i:ch for i,ch in enumerate(vocab) }
stoi

In [ ]:
from mingpt.model import GPT
model_config = GPT.get_default_config()
model_config.model_type = 'gpt2'
model_config.vocab_size =len(vocab) # openai's model vocabulary
model_config.block_size = 1024 


model = GPT(model_config)

model

In [ ]:

class Dataset(torch.utils.data.Dataset):

  def __init__(self, examples,sizeBlock):
    self.examples = examples
    self.sizeBlock = sizeBlock

  def __len__(self):
    return len(self.examples)
   
  def __getitem__(self, ix):
    dato=self.examples[ix]
    entrada=dato[:-1]
    salida = dato[1:]
    input = np.pad(entrada, (0,self.sizeBlock-len(entrada)), 'constant', constant_values=0)
    target = np.pad(salida, (0,self.sizeBlock-len(salida)), 'constant', constant_values=0)
    assert len(input) == self.sizeBlock
    assert len(target) == self.sizeBlock

    return torch.LongTensor(input), torch.LongTensor(target)

In [ ]:
instanciaDataset=Dataset([datoPrueba],1024)
input,target=instanciaDataset[0]
# print(input)


In [ ]:
# print(target)

In [ ]:
dl = torch.utils.data.DataLoader(instanciaDataset, batch_size=1, shuffle=False)

x, y = next(iter(dl))
x.shape, y.shape

(torch.Size([1, 1024]), torch.Size([1, 1024]))

In [ ]:
!pip install pytorch_lightning

In [ ]:
import pytorch_lightning as pl  

class Module(pl.LightningModule):
	def __init__(self, model, tconf):
		super().__init__()
		self.model = model 
		self.tconf = tconf

	def forward(self, x):
		return self.model(x)[0]

	def training_step(self, batch, batch_idx):
		y, loss = self.model(*batch)
		return loss 

	def validation_step(self, batch, batch_idx):
		y, loss = self.model(*batch)
		self.log('val_loss', loss.item())

	def configure_optimizers(self):
		return self.model.configure_optimizers(self.tconf)

In [ ]:
from mingpt.trainer import Trainer
train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # many possible options, see the file
train_config.batch_size=1
module=Module(model,train_config)

In [ ]:
trainer = pl.Trainer(
	gpus=1,
	max_epochs=1000,
	precision=16,
	overfit_batches=1,
	logger=False,
	enable_checkpointing=False,
)

trainer.fit(module, dl)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(overfit_batches=1)` was configured so 1 batch will be used.
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:110: PossibleUserWarning: 

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1000` reached.


In [ ]:
model.eval()

In [ ]:
new_words = 15
entr=input[:300].expand(1,-1)
y = model.generate(entr, new_words)
#Tokens generados
y 

tensor([[  1,  45,  34,  41, 103,   3, 107,   4, 107,   3,  19, 107,   3, 107,
           4, 105,   3, 107,   4, 105,   4, 107,   3, 105,   3, 107,   3, 109,
           3, 110,   8,  20,   4, 102,   3, 105,   4, 105,   3, 105,   3, 105,
           4, 107,   3, 105,   5, 100,   3, 102,   3, 100,   3,  99,   3, 102,
           3, 100,   6,  20,   6,  24,  23, 100,   3, 103,   4, 103,   3, 103,
           3, 103,   4, 102,   3, 103,   4, 102,   4, 103,   3, 102,   3, 103,
           3, 105,   3, 107,   8,  20,   4, 102,   3, 105,   4, 105,   3, 105,
           3, 105,   4, 107,   3, 105,   5, 102,   3, 102,   3, 100,   3,  99,
           3, 102,   3,  21, 100,   8,  20,   4,  24,  21, 100,   5, 107,   3,
         107,   3, 107,   4, 105,   3,  21,  23, 107,   3, 107,   4, 105,   3,
         107,   5, 105,   3, 107,   8,  20,   4, 102,   3, 105,   4, 105,   3,
         105,   3, 105,   4, 107,   3, 105,   5, 102,   3, 102,   3, 100,   3,
          99,   3, 102,   3,  21, 100,   5, 107,   3

In [ ]:
#Token real partitura
print(target[:315])

tensor([ 45,  34,  41, 103,   3, 107,   4, 107,   3,  19, 107,   3, 107,   4,
        105,   3, 107,   4, 105,   4, 107,   3, 105,   3, 107,   3, 109,   3,
        110,   8,  20,   4, 102,   3, 105,   4, 105,   3, 105,   3, 105,   4,
        107,   3, 105,   5, 100,   3, 102,   3, 100,   3,  99,   3, 102,   3,
        100,   6,  20,   6,  24,  23, 100,   3, 103,   4, 103,   3, 103,   3,
        103,   4, 102,   3, 103,   4, 102,   4, 103,   3, 102,   3, 103,   3,
        105,   3, 107,   8,  20,   4, 102,   3, 105,   4, 105,   3, 105,   3,
        105,   4, 107,   3, 105,   5, 102,   3, 102,   3, 100,   3,  99,   3,
        102,   3,  21, 100,   8,  20,   4,  24,  21, 100,   5, 107,   3, 107,
          3, 107,   4, 105,   3,  21,  23, 107,   3, 107,   4, 105,   3, 107,
          5, 105,   3, 107,   8,  20,   4, 102,   3, 105,   4, 105,   3, 105,
          3, 105,   4, 107,   3, 105,   5, 102,   3, 102,   3, 100,   3,  99,
          3, 102,   3,  21, 100,   5, 107,   3, 107,   3, 107,  

In [ ]:
partitura=[ 45,  34,  41, 103,   3, 107,   4, 107,   3,  19, 107,   3, 107,   4,
        105,   3, 107,   4, 105,   4, 107,   3, 105,   3, 107,   3, 109,   3,
        110,   8,  20,   4, 102,   3, 105,   4, 105,   3, 105,   3, 105,   4,
        107,   3, 105,   5, 100,   3, 102,   3, 100,   3,  99,   3, 102,   3,
        100,   6,  20,   6,  24,  23, 100,   3, 103,   4, 103,   3, 103,   3,
        103,   4, 102,   3, 103,   4, 102,   4, 103,   3, 102,   3, 103,   3,
        105,   3, 107,   8,  20,   4, 102,   3, 105,   4, 105,   3, 105,   3,
        105,   4, 107,   3, 105,   5, 102,   3, 102,   3, 100,   3,  99,   3,
        102,   3,  21, 100,   8,  20,   4,  24,  21, 100,   5, 107,   3, 107,
          3, 107,   4, 105,   3,  21,  23, 107,   3, 107,   4, 105,   3, 107,
          5, 105,   3, 107,   8,  20,   4, 102,   3, 105,   4, 105,   3, 105,
          3, 105,   4, 107,   3, 105,   5, 102,   3, 102,   3, 100,   3,  99,
          3, 102,   3,  21, 100,   5, 107,   3, 107,   3, 107,   4, 105,   3,
         24,  21, 100,   4, 100,   4, 100,   3,  98,   3,  95,   4,  21,  23,
        112,   3, 112,   3, 110,   3, 110,   3, 107,   3, 107,   3, 105,   3,
        107,   3, 110,   8,  20,   4, 105,   3, 107,   3, 110,   3, 105,   3,
        107,   3, 110,   3, 105,   3, 103,   3, 100,   8,  20,   4,  24,  23,
        100,   3, 103,   4, 103,   3, 103,   3, 103,   4, 102,   3, 103,   4,
        102,   4, 103,   3, 102,   3, 103,   3, 105,   3, 107,   8,  20,   4,
        102,   3, 105,   4, 105,   3, 105,   3, 105,   4, 107,   3, 105,   5,
        102,   3, 102,   3, 100,   3,  99,   3, 102,   3,  21, 100,   8,  20,
          4,  24,  21, 100,   4, 100,   4, 100,   3,  98,   3,  95,   4,  22,]

In [ ]:
#Apartir de esta celda el siguiente codigo es para graficar los tokens a una imagen de partitura lo cual no se pudo terminar en este Sprint
partituraLimpia,vocabReves=convertir_tokens_a_notas(vocabulario=vocab,listTokens=partitura)
partituraLimpia=np.array(partituraLimpia)
vocabReves[partitura[5]]

'81'

In [ ]:
#permite que music21 renderice imágenes de notas musicales
print('installing Lilypond...')
!apt-get install lilypond > /dev/null

# convierte archivos midi en archivos wav para poder reproducirlos
print('installing fluidsynth...')
!apt-get install fluidsynth > /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2


print('done!')

installing Lilypond...
Extracting templates from packages: 100%
installing fluidsynth...
done!


In [ ]:
import music21
from IPython.display import Image, Audio
def show(music):
  display(Image(str(music.write('lily.png'))))

def play(music):
  filename = music.write('mid')
  !fluidsynth -ni font.sf2 $filename -F $filename\.wav -r 44100 > /dev/null
  display(Audio(filename + '.wav'))

In [ ]:

from music21 import *

def graficarPartitura(notas,titulo,vocabReves):
  score = stream.Score()
  verse = stream.Part()
  measure = stream.Measure()
  score.insert(0, metadata.Metadata())
  score.metadata.composer = 'Aland Bravo Vecorena'
  
  
  for i in range(0,len(notas)):
    acumNotas=None
    if notas[i] in range(25,39+1):
      song_key = key.Key(key.sharpsToPitch(int(vocabReves[notas[i]]))) #asigna el numero de # o bemol
      measure.insert(0, song_key)
      song_key.octave = 4 # moverse hasta la 4ta octava alrededor del C central
      score.metadata.title = titulo+" %s" % song_key.name
    if notas[i] in range(40,44+1):
      verse.append(meter.TimeSignature(vocabReves[notas[i]]))#asigna el compas 2/4
    if notas[i] in range(47,134+1):
      if notas[i+1]in range(3,18+1):
        if notas[i+2] ==19:
          acumNotas=note.Note(int(vocabReves[notas[i]]),quarterLength=float(vocabReves[notas[i+1]]))
          acumNotas.tie=tie.Tie('start')
        else:
          acumNotas=note.Note(int(vocabReves[notas[i]]),quarterLength=float(vocabReves[notas[i+1]]))
    if notas[i]==20:
      if notas[i+1]in range(3,18+1):
        acumNotas=note.Rest(quarterLength=float(vocabReves[notas[i+1]]))
    if notas[i]==21:
     measure.append(bar.Barline(style='regular'))
    if notas[i]==22:
      measure.append(bar.Barline(style='final'))
    if notas[i]==23:
      measure.leftBarline = bar.Repeat(direction='start')
    if notas[i]==24:
      measure.rightBarline = bar.Repeat(direction='end')
    if acumNotas != None:
      measure.append(acumNotas)
  
  verse.append(measure)
  score.append(verse)
  score.show('text')
  play(score)
  show(score)






In [ ]:
graficarPartitura(partitura,"nombre",vocabReves)